# 1D spectral analysis and light curve calculation from DL3 (energy-dependent angular cuts)

Originally created by Chaitanya Priyadarshi and adapted to work with Gammapy v1.1 and the latest DL3 files produced by lstchain v0.10.

This is a combination of Gammapy tutorials for the **1D spectral analysis following the ON-OFF forward-folding method and light curve calculation** for point-like-source observations in wobble mode.

Original notebooks can be accessed at:

 - https://docs.gammapy.org/1.1/tutorials/analysis-1d/spectral_analysis_rad_max.html (from DL3 with energy-dependent angular cuts, the *approach followed in this notebook*)
 - https://docs.gammapy.org/1.1/tutorials/analysis-1d/spectral_analysis.html (from DL3 with global gammaness and angular cuts)
 - https://docs.gammapy.org/1.1/tutorials/analysis-time/light_curve.html
 - https://docs.gammapy.org/1.1/tutorials/analysis-time/light_curve_flare.html


It reduces a set of DL3 files (with energy-dependent angular cuts) into an energy-binned dataset, fits a spectral model to this dataset, calculates spectral flux points and computes the light curve.

Here it is also described how to write all these objects to files, read them back, inspect and plot the results.

-----------
## Content

### 0. Inspect the DL3 file content
### 1. Read the DL3 index files and load the data
### 2. Selection filters for the observations
### 3. Define Target position and energy ranges for reconstructed events

### 4. Define the base Map geometries for creating the SpectrumDataset

### 5. Data reduction chain

### 6. Generate the Spectrum Dataset for all observations

### 7. Some plots with the given Dataset

### 8. Write all datasets into OGIP files
### 9. Get Pivot energy to fix the reference energy and define the Spectrum Model
### 10. Spectral Fitting
### 11. Check the Flux points
### 12. SED plots
### 13. Light curve
### 14. Save the SED and LC Flux Points and Model to separate files

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

import numpy as np
from regions import CircleSkyRegion, PointSkyRegion
from pathlib import Path
import os
import pickle

In [ ]:
from gammapy.data import DataStore, EventList
from gammapy.datasets import (
    Datasets,
    FluxPointsDataset,
    SpectrumDataset,
    SpectrumDatasetOnOff,
)
from gammapy.estimators import FluxPointsEstimator, LightCurveEstimator, FluxPoints
from gammapy.makers import (
    ReflectedRegionsBackgroundMaker,
    SafeMaskMaker,
    SpectrumDatasetMaker,
    WobbleRegionsFinder
)
from gammapy.maps import MapAxis, RegionGeom, WcsGeom, Map
from gammapy.modeling import Fit
from gammapy.modeling.models import (
    PowerLawSpectralModel,
    LogParabolaSpectralModel,
    create_crab_spectral_model,
    SkyModel,
)
from gammapy.visualization import plot_spectrum_datasets_off_regions

from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy.table import Table
from astropy.time import Time
import astropy.units as u

In [ ]:
# Define the path to the DL3 files
base_dir = Path("/fefs/aswg/workspace/analysis-school-2024")
dl3_path = base_dir / "DL3/Crab_Dec_2023"

# 0. Inspect the DL3 file content

In [ ]:
# Open one of the DL3 files from the directory
filename = dl3_path / "dl3_LST-1.Run15996.fits"
events = EventList.read(filename)

In [ ]:
# Have a look at the event list (here the first 10 events)
events.table[:10]

In [ ]:
# You can have a look of the events with
events.peek()

In [ ]:
# You can select events based any parameter on the table above, e.g. energy:
selected_energy = events.select_energy([500 * u.GeV, 1 * u.TeV])
selected_energy.peek()

In [ ]:
# or gammaness:
gh_range = [0.9, 1]
selected_events_gh = events.select_parameter(parameter="GAMMANESS", band=gh_range)
selected_events_gh.peek()

# 1. Read the DL3 index files and load the data

Let's load now all the DL3 files together.
If the DL3 index files are not present, run the `lstchain_create_dl3_index_files` for the given DL3 files.

`lstchain_create_dl3_index_files -d $dl3_path`  (by default create the index files in the same directory)

In [ ]:
total_datastore = DataStore.from_dir(dl3_path)

ogip_path = dl3_path / "OGIP"

# Create the Paths if they do not exist already
ogip_path.mkdir(exist_ok=True, parents=True)

In [ ]:
# Let's have a first look at the first 5 entries of the table from all observations present in the index files. 
# It contains run-wise information.
total_datastore.obs_table

In [ ]:
total_datastore.obs_table['LIVETIME']

In [ ]:
# Also you can have a look at the HDU table, which contains information on the data 
# and instrument response function files for each observation
total_datastore.hdu_table[:6]

# 2. Selection filters for the observations

Based on the run-wise information from the previous table, filters can be applied to select just a subset of the observation list based on source name, zenith angle or livetime.

This list of observations based on similar filters can be also obtained beforehand by using the [data quality notebook](https://github.com/cta-observatory/cta-lstchain/blob/main/notebooks/data_quality.ipynb) (recommended). Then you can directly pass this list of `obs_id` to `get_observations` method of `DataStore` object below.

In [ ]:
# Get the object name from the OBS Table, assuming all the DL3 files are of the same single source.
# If not, then select a single object, to produce the relevant Spectrum Dataset

obj_name = np.unique(total_datastore.obs_table["OBJECT"])[0]
print("The source is", obj_name)

max_zen = 60  # in deg for a maximum limit on zenith pointing of observations
min_time = 0  # in seconds for minimum livetime of each observation

In [ ]:
total_obs_list = total_datastore.obs_table["OBS_ID"].data
observations_total = total_datastore.get_observations(
    total_obs_list, 
    required_irf=["aeff", "edisp", "rad_max"],  # By default, "full-enclosure" : ["events", "gti", "aeff", "edisp", "psf", "bkg"]
                        # If not all IRFs are present, the entry will be skipped 
    skip_missing=False # Skip missing observations, within the list provided earlier
)

In [ ]:
d_time = total_datastore.obs_table["LIVETIME"] > min_time
d_zen = total_datastore.obs_table["ZEN_PNT"] < max_zen
d_obj = total_datastore.obs_table["OBJECT"] == obj_name

obs_table_selected = total_datastore.obs_table[d_zen & d_obj & d_time]
obs_id_list = obs_table_selected["OBS_ID"]

observations_sel = total_datastore.get_observations(
    obs_id_list, 
    required_irf="point-like"
)

In [ ]:
print('Observation runs selected are:', obs_id_list.data)
print(f'Total livetime of all observations: {total_datastore.obs_table["LIVETIME"].to(u.h).sum():.1f}')
print(f'Total livetime of selected observations {obs_table_selected["LIVETIME"].data.sum()/3600:.1f} hrs')

# 3. Define Target position and energy ranges for reconstructed events

In [ ]:
target_position = SkyCoord.from_name("Crab", frame='icrs')
target_position

If we were using global theta cut, this is the way of getting the theta cut (`RAD_MAX` key) used for the IRF production.

Here we will use energy-dependent cuts, therefore the following is not executed in this example.

```
# Find the fixed global theta cut used for creating the IRFs

theta_cut = observations_sel[0].aeff.meta["RAD_MAX"]
print("Theta cut applied for creating the IRF in the selected DL3 file,", theta_cut)

# Converting the value into astropy.units to be used for defining the ON region with CircleSkyRegion
on_region_radius = u.Quantity(theta_cut)
```

In [ ]:
# The metadata of the DL3 contains the efficiency of the cuts used for the DL3 production.
observations_sel[0].aeff.meta

In [ ]:
# Provide the minimum, maximum energies in TeV units, and number of bins per decade, to create the 
# required reconstructed and true energy ranges.
# For Light Curve estimation and spectral fitting, flux calculation can be only performed within 
# the energy edges provided for the reconstructed events.
# For example, if the reconstructed energy edges are [0.01, 0.1, 1, 10] TeV and you want LC in 
# [0.05, 10] TeV energy range, then, reproduce the Dataset objects with those reconstructed energy edges.

e_reco_min = 0.05 # 0.01
e_reco_max = 50

e_true_min = 0.01
e_true_max = 100

# Using bins per decade
e_reco_bin_p_dec = 8
e_true_bin_p_dec = 10

energy_axis = MapAxis.from_energy_bounds(
    e_reco_min, e_reco_max, 
    nbin=e_reco_bin_p_dec, per_decade=True, 
    unit="TeV", name="energy"
)
energy_axis_true = MapAxis.from_energy_bounds(
    e_true_min, e_true_max, 
    nbin=e_true_bin_p_dec, per_decade=True, 
    unit="TeV", name="energy_true"
)

# Select minimum and maximum energy edges for the SED, from the energy_axis to be used un the Dataset
# Here we use a different minimum energy than energy_axis, but the same energy bins.
# For analyzers who do not want energy bins per decade, or some custom bins for energy_axis, 
# make appropriate changes in each axis.
e_fit_min = energy_axis.edges[1].value
e_fit_max = energy_axis.edges[-1].value
e_fit_bin_p_dec = e_reco_bin_p_dec

# Just to have a separate MapAxis for spectral fit energy range
energy_fit_edges = MapAxis.from_energy_bounds(
    e_fit_min, e_fit_max, 
    nbin=e_fit_bin_p_dec, per_decade=True, 
    unit="TeV"
).edges


print("Spectral Fit will be done in energy edges:\n", energy_fit_edges)

In [ ]:
# You can access the energy edges and center values
energy_axis_true.edges

# 4. Define the base Map geometries for creating the SpectrumDataset

In [ ]:
on_region = PointSkyRegion(target_position)  

# This will create the base geometry in which to bin the events based on their reconstructed positions
on_geom = RegionGeom.create(
    on_region, 
    axes=[energy_axis]
)
# In case of using global angular cut `CircleSkyRegion` should be used instead (see "Spectral analysis Gammapy tutorial")

# 5. Data Reduction chain
Create some Dataset and Data Reduction Makers

In [ ]:
# geom is the target geometry in reco energy for counts and background maps
# energy_axis_true is the true energy axis for the IRF maps
dataset_empty = SpectrumDataset.create(
    geom=on_geom, 
    energy_axis_true=energy_axis_true
)
# When not including a PSF IRF, put the containment_correction as False
dataset_maker = SpectrumDatasetMaker(
    containment_correction=False, 
    selection=["counts", "exposure", "edisp"]
)

In [ ]:
# The following makers can be tuned and played to check the final Dataset to be used.

# In the case of energy-dependent angular cuts we have to use the `WobbleRegionsFinder`, 
# to determine the OFF positions, depending on the number of regions specified.
# Their sizes will be defined by the theta values in RAD_MAX_2D table based on the estimated energy binning.
# The same logic applies to the size of the ON region.


# Background maker will use the WobbleRegionsFinder, assuming 1 OFF region for the background estimation
region_finder = WobbleRegionsFinder(n_off_regions=1)
bkg_maker = ReflectedRegionsBackgroundMaker(region_finder=region_finder)

In [ ]:
# Maker for safe energy range for the events.
safe_mask_maker = SafeMaskMaker(
    methods=["aeff-max"], 
    aeff_percent=5
)

# Or make a custom safe energy range for the events.
#safe_min_energy = 50 * u.GeV
#safe_max_energy = 20 * u.TeV
# For other arguments and options, check the documentation,
# https://docs.gammapy.org/1.1/api/gammapy.makers.SafeMaskMaker.html#gammapy.makers.SafeMaskMaker

# 6. Generate the Spectrum Dataset for all observations

In [ ]:
%%time
# The final object will be stored as a Datasets object

# There will be an error message on use_region_center=False. It is a bug message in gammapy, so just ignore it
datasets = Datasets()

# Create a counts map for visualisation later
counts = Map.create(skydir=target_position, width=3)

for obs_id, observation in zip(obs_id_list, observations_sel):
    dataset = dataset_maker.run(
        dataset_empty.copy(name=str(obs_id)), 
        observation
    )
    print('obs_id:', obs_id)
    dataset_on_off = bkg_maker.run(
        dataset=dataset, 
        observation=observation
    )
    counts.fill_events(observation.events)
    
    # Check the LC and SEDs by applying the safe mask to see the distinction.
    dataset_on_off = safe_mask_maker.run(dataset_on_off, observation)
    
    # Or use custom safe energy range
    #dataset_on_off.mask_safe = dataset_on_off.counts.geom.energy_mask(
    #    energy_min=safe_min_energy, energy_max=safe_max_energy
    #)
    
    datasets.append(dataset_on_off)    

In [ ]:
print(datasets[0])

# 7. Some plots with the given Dataset

In [ ]:
len(datasets)

In [ ]:
# Check the target position and OFF regions used for the calculation of the excess
ax = counts.plot(cmap="viridis", stretch="sinh")
on_geom.plot_region(ax=ax, kwargs_point={"color": "k", "marker": "*"})
plot_spectrum_datasets_off_regions(ax=ax, datasets=datasets)
plt.show()

In [ ]:
info_table = datasets.info_table(cumulative=True)
info_table

In [ ]:
# Plot temporal evolution of excess events and significance value
plt.figure(figsize=(10,5))
plt.subplot(121)
plt.plot(
    info_table["livetime"].to("h"), info_table["excess"], marker="o", ls="none"
)
plt.plot(info_table["livetime"].to("h")[-1:1], info_table["excess"][-1:1], 'r')
plt.xlabel("Livetime (h)")
plt.ylabel("Excess")
plt.grid()
plt.title('Excess vs Livetime')

plt.subplot(122)
plt.plot(
    np.sqrt(info_table["livetime"].to("h")),
    info_table["sqrt_ts"],
    marker="o",
    ls="none",
)
plt.grid()
plt.xlabel("Sqrt Livetime h^(1/2)")
plt.ylabel("sqrt_ts")
plt.title('Significance vs Square root of Livetime')
plt.subplots_adjust(wspace=0.5)


In [ ]:
%%time
# Plot the counts+excess, exposure and energy migration of each selected dataset
for data in datasets:
    plt.figure(figsize=(21, 5.5))
    plt.subplot(131)
    data.plot_counts()
    data.plot_excess()
    plt.grid(which="both")
    plt.title(f'Run {data.name} Counts and Excess')
    
    plt.subplot(132)
    data.exposure.plot()
    plt.grid(which='both')
    plt.title(f'Run {data.name} Exposure')
    
    plt.subplot(133)
    if data.edisp is not None:
        kernel = data.edisp.get_edisp_kernel()
        kernel.plot_matrix(add_cbar=True)
        plt.title(f'Run {data.name} Energy Dispersion')
    plt.subplots_adjust(wspace=0.3)
    plt.show()
    plt.close()

# 8. Write all datasets into OGIP files

In [ ]:
for d in datasets:
    d.write(
        filename=ogip_path / f"obs_{d.name}.fits.gz", overwrite=True
    )

In [ ]:
# Read the OGIP files to include the source object name in its headers, to be used for further analysis
for obs in obs_id_list:
    file = ogip_path/f"obs_{obs}.fits.gz"
    
    d1 = fits.open(file)
    d1.writeto(file, overwrite=True)

# 9. Get Pivot energy to fix the reference energy and define the Spectrum Model

In [ ]:
# Find pivot (decorrelation) energy for a Power Law model to get the reference energy for Log Parabola model
def get_pivot_energy(datasets, e_ref, e_edges, obj_name):
    """
    Using Power Law spectral model with the given reference energy and 
    get the decorrelation energy of the fit, within the fit energy range, e_edges.
    This method is further explained in doi:10.1088/0004-637X/707/2/1310
    """
    spectral_model = PowerLawSpectralModel(
        index=2, amplitude=2e-11 * u.Unit("cm-2 s-1 TeV-1"), reference=e_ref
    )
    model = SkyModel(spectral_model=spectral_model, name=obj_name)
    model_check = model.copy()

    # Stacked dataset method
    stacked_dataset = Datasets(datasets).stack_reduce()
    stacked_dataset.models = model_check

    fit_stacked = Fit()
    result_stacked = fit_stacked.run(datasets=stacked_dataset)

    return model_check.spectral_model.pivot_energy


In [ ]:
# Using a reference energy close to the expected decorrelation energy
ref = get_pivot_energy(datasets, 0.4 * u.TeV, energy_axis.edges, obj_name)
print(ref.to_value(u.GeV))

In [ ]:
# Final spectral model of Log Parabola, to be used for estimating the LC.
# One can try different Spectral Models as well.
# Be careful in the choice of Spectral Model being used for the 2 examples presented here

# Crab
spectral_model_lp = LogParabolaSpectralModel(
        amplitude = 5e-12 * u.Unit('cm-2 s-1 TeV-1'),
        reference = ref,
        alpha = 2 * u.Unit(''),
        beta = 0.1 * u.Unit('')
)
model_lp = SkyModel(spectral_model=spectral_model_lp, name=obj_name)


In [ ]:
# Use the appropriate models, as per the selection of the source/dataset
params=model_lp.to_dict()['spectral']
params

# 10. Spectral Fitting
One can check for a more comprehensive tutorial on Modelling and Fitting in the Gammapy tutorials
* https://docs.gammapy.org/1.1/tutorials/api/fitting.html
* https://docs.gammapy.org/1.1/tutorials/api/model_management.html

In [ ]:
# Using stacked analysis method, where we stack together all Datasets into 1 Dataset and add the model afterwards
stacked_dataset = Datasets(datasets).stack_reduce()
stacked_dataset.models = model_lp

In [ ]:
# Fitting the model to the dataset
fit = Fit()
result = fit.run(datasets=stacked_dataset)
model_best = model_lp

In [ ]:
model_best.parameters[2]

In [ ]:
# Compute the Flux Points after Fitting the model
# We do not do too many optimizations here. 
# If one wants, can try and check the various attributes of the Estimator
fpe = FluxPointsEstimator(
    energy_edges=energy_fit_edges, 
    reoptimize = False, # Re-optimizing other free model parameters (not belonging to the source)
    source=obj_name,
    selection_optional="all" # Estimates asymmetric errors, upper limits and fit statistic profiles
)
flux_points = fpe.run(datasets=stacked_dataset)

flux_points_dataset = FluxPointsDataset(
    data=flux_points, models=model_best
)

In [ ]:
result

In [ ]:
model_best.to_dict()['spectral']['parameters']

# 11. Check the Flux points

In [ ]:
# Check the Flux table
# sed_type options are {“likelihood”, “dnde”, “e2dnde”, “flux”, “eflux”} with "likelihood" being default
# format options are {“gadf-sed”, “lightcurve”, “binned-time-series”, “profile”} with "gadf-sed" being default
flux_points.to_table(formatted=True, sed_type="e2dnde")

In [ ]:
# Fit Statistic array
print(flux_points_dataset.stat_array())

# Total statistics sum
print(flux_points_dataset.stat_sum(), np.nansum(flux_points_dataset.stat_array()))

In [ ]:
# Add the Fit results to the model object to save it to file
model_final = model_best.to_dict()
model_final["FitResult"] = {
    "Optimize": result.optimize_result,
    "Covariance": result.covariance_result
}

In [ ]:
model_final

In [ ]:
model_best.parameters.to_table()

# 12. SED Plots

In [ ]:
ref_label="Crab MAGIC LP (JHEAp 2015)"

In [ ]:
flux_points.to_table(formatted=True, sed_type="e2dnde")

In [ ]:
# Setting plot axes limits and other args

# Here we plot only in the energy range from where we have the first flux point, and before the first UL.
# This is based on the selection we used earlier on the dataset and the Flux points estimation we get.
# One should adjust the limit as per selections used earlier.

positive_flux = flux_points.to_table(formatted=True, sed_type="e2dnde")["e2dnde"] > 0
e_plot_min = flux_points.to_table(formatted=True, sed_type="e2dnde")["e_min"].quantity[positive_flux][0]

non_ul = flux_points.to_table(formatted=True, sed_type="e2dnde")["is_ul"] == 0
e_plot_max = flux_points.to_table(formatted=True, sed_type="e2dnde")["e_max"].quantity[non_ul][-1]

sed_kwargs = {
    "sed_type": "e2dnde",
    "energy_bounds": [e_plot_min, e_plot_max],
}

# Using the energy range used in the MAGIC reference
ds_magic_ref_kwargs = {
    "sed_type": "dnde",
    "energy_bounds": [50 * u.GeV, 30 * u.TeV],
}
sed_magic_ref_kwargs = {
    "sed_type": "e2dnde",
    "energy_bounds": [50 * u.GeV, 30 * u.TeV],
}
sed_plot_kwargs = {
    "label": "LST-1 data",
}
plot_ts_kwargs = {
    "color": "darkorange"
}

In [ ]:
# Calculate & plot Crab reference flux
# https://doi.org/10.1016/j.jheap.2015.01.002
crab = create_crab_spectral_model("magic_lp")
crab.amplitude.error = 0.03e-11 * u.Unit("cm-2 s-1 TeV-1")
crab.alpha.error = 0.01
crab.beta.error = 0.01/np.log(10)


In [ ]:
plt.figure(figsize=(8,5))
ax = flux_points.plot(sed_type="e2dnde", **plot_ts_kwargs)

flux_points.plot_ts_profiles(ax=ax, sed_type="e2dnde")
plt.xlim(e_plot_min.value, e_plot_max.value)

plt.grid(which='both')
plt.title('TS Profiles')

In [ ]:
# Fit model covariance matrix plot
model_best.covariance.plot_correlation()

In [ ]:
fig_sed = plt.figure(figsize=(8,8))

gs2 = GridSpec(7, 1)

gs2.update(hspace=0.1)
args1 = [gs2[:5,:]]
args2 = [gs2[5:,:]]

fig_gs1 = fig_sed.add_subplot(*args1)
fig_gs2 = fig_sed.add_subplot(*args2)

FluxPointsDataset(data=flux_points, models=model_best).plot_spectrum(
    ax=fig_gs1, 
    kwargs_fp=sed_plot_kwargs, 
)

create_crab_spectral_model("magic_lp").plot(
    ax=fig_gs1, **sed_magic_ref_kwargs, label=ref_label
)

fig_gs1.legend()
fig_gs1.set_xlim(e_plot_min.value, e_plot_max.value)
fig_gs1.set_ylim(2e-12, 2e-10)
fig_gs1.tick_params(labelbottom=False)

fig_gs1.grid(which='both')
fig_gs1.set_title('SED')

flux_points_dataset.plot_residuals(ax=fig_gs2, method='diff/model')
fig_gs2.grid(which='both')
fig_gs2.set_xlim(e_plot_min.value, e_plot_max.value);

In [ ]:
fig = plt.figure(figsize=(8,7))
gs = GridSpec(7, 1)

args1 = [gs[:5,:]]
args2 = [gs[5:,:]]
kwargs_res = {"method": "diff/sqrt(model)"}

fig_gs1 = fig.add_subplot(*args1)
fig_gs2 = fig.add_subplot(*args2)

stacked_dataset.plot_excess(fig_gs1)
fig_gs1.grid(which="both")
fig_gs1.set_ylabel("Excess")

stacked_dataset.plot_residuals_spectral(fig_gs2, **kwargs_res, region=stacked_dataset.counts.geom.region)
fig_gs2.grid(which="both")

fig_gs2.set_ylabel(f"Residuals \n (data-model)/sqrt(model)")

In [ ]:
plt.figure(figsize=(8,5))

flux_points.plot(sed_type="dnde", label='Joint flux')
create_crab_spectral_model("magic_lp").plot(**ds_magic_ref_kwargs, label=ref_label)
plt.xlim(e_plot_min.value, e_plot_max.value)
plt.grid(which='both')
plt.legend()
plt.title('Differential spectrum')

# 13. Light curve

In [ ]:
# Define energy range for the light curve. It has to match the edges of the 
# reconstructed energy binning set for the SED! :
e_lc_min = energy_axis.edges[5]
e_lc_max = energy_axis.edges[-1]
print("LC will be estimated from ", e_lc_min, "to ", e_lc_max)

# Plotting settings
lc_kwargs = {
    "marker": "o", 
    "label": "LST-1",
}


In [ ]:
# Get the GTI parameters of each observation to create time intervals for plotting LC
t_start = []
t_stop = []
tot_time = []

for obs in observations_sel:
    gti = obs.gti
    
    t_start.append(gti.time_start[0])
    t_stop.append(gti.time_stop[0])
    tot_time.append(gti.time_sum.value)

t_start = np.sort(np.array(t_start))
t_stop = np.sort(np.array(t_stop))
tot_time = np.array(tot_time)

t_start = Time(t_start)
t_stop = Time(t_stop)

t_day = np.unique(np.rint(t_start.mjd))

# To make the range night-wise, keep the MJD range in half-integral values
t_range = [Time([t-0.5, t+0.5], format="mjd", scale="utc") for t in t_day]

In [ ]:
# Create the LC Estimator on a run-by-run basis and nightly
lc_maker_1d = LightCurveEstimator(
    energy_edges=[e_lc_min, e_lc_max], 
    reoptimize=False, # Re-optimizing other free model parameters (not belonging to the source)
    source=obj_name, 
    selection_optional="all" # Estimates asymmetric errors, upper limits and fit statistic profiles
)

lc_maker_night_wise = LightCurveEstimator(
    energy_edges=[e_lc_min, e_lc_max], 
    time_intervals=t_range,
    reoptimize=False, 
    source=obj_name,
    selection_optional="all"
)

# Assigning the best fit model for each dataset
for data in datasets:
    data.models = model_best

In [ ]:
lc_1d = lc_maker_1d.run(datasets)
lc_night = lc_maker_night_wise.run(datasets)

In [ ]:
# If there are more than 1 night of data, one can see the integrated light curve for each night
lc_night.to_table(sed_type="flux", format="lightcurve")

In [ ]:
# Check the various column data of the Light Curve object
lc_1d.to_table(sed_type="flux", format="lightcurve")

In [ ]:
# Calculate Crab reference flux
flux_crab, flux_crab_error = crab.integral_error(e_lc_min, e_lc_max)
print(flux_crab, flux_crab_error)

In [ ]:
fig_lc = plt.figure(figsize=(8,10))

gs2 = GridSpec(10, 5)

gs2.update(wspace=0.4)
args1 = [gs2[:5,:]]
args2 = [gs2[5:,:]]

fig_gs1 = fig_lc.add_subplot(*args1)
fig_gs2 = fig_lc.add_subplot(*args2, sharey=fig_gs1)

lc_1d.plot(
    ax=fig_gs1,
    sed_type="flux",
    **lc_kwargs
)
fig_gs1.axhline(
    flux_crab.to_value("cm-2 s-1"), c='red', ls='--', 
    label='Crab (MAGIC, JHEAp 2015)'
)
fig_gs1.axhspan(
    (flux_crab - flux_crab_error).to_value("cm-2 s-1"), 
    (flux_crab + flux_crab_error).to_value("cm-2 s-1"), 
    alpha=0.2, color='tab:orange'
)
fig_gs1.get_xaxis().set_ticklabels([])
fig_gs1.grid(which='both')
fig_gs1.set_title(
    f'LC LST-1 {obj_name}: {e_lc_min.to(u.GeV):.0f} < E < {e_lc_max.to(u.TeV):.1f} \nRun-wise {tot_time.sum()/3600:.1f} hrs, night-wise {len(t_day)} nights'
)
fig_gs1.legend()
fig_gs2.set_yscale('linear')

# Custom y-axis range, if needed:
#fig_gs2.set_ylim(0, 4e-10)
#fig_gs2.set_ylim(0, 5e-11)

fig_gs2.set_ylim(0, 1.5*np.nanmax(lc_1d.flux.data))

fig_gs1.get_yaxis().get_offset_text().set_position((-0.06,1))

lc_night.plot(
    ax=fig_gs2,
    sed_type="flux",
    axis_name="time",
    **lc_kwargs
)
fig_gs2.axhline(
    flux_crab.to_value("cm-2 s-1"), c='red', ls='--', 
    label='Crab (MAGIC, JHEAp 2015)'
)
fig_gs2.axhspan(
    (flux_crab - flux_crab_error).to_value("cm-2 s-1"), 
    (flux_crab + flux_crab_error).to_value("cm-2 s-1"), 
    alpha=0.2, color='tab:orange'
)

fig_gs2.grid(which='both')
fig_gs2.set_yscale('linear')
fig_gs2.legend()

# 14. Save the SED and LC Flux Points and Model to separate files
## This way, one can plot different SEDs and LCs together

In [ ]:
# Dump the model and Optimization results into a file
f = open(dl3_path / f'{obj_name}_dataset_{datasets[0].name}_to_{datasets[-1].name}_flux_model_dict.dat', 'wb')
pickle.dump(model_final, f)
f.close()

In [ ]:
# Create a temporary fits file with the SED and LC Flux Points tables
f = fits.HDUList(
    [
        fits.PrimaryHDU(),
        fits.BinTableHDU(flux_points.to_table(), name="SED"),
        fits.BinTableHDU(lc_1d.to_table(sed_type="flux", format="lightcurve"), name="LC"),
    ]
)
f.writeto(
    dl3_path / f'{obj_name}_dataset_{datasets[0].name}_to_{datasets[-1].name}_flux_pts.fits', 
    overwrite=True
)

### Example on reading back and plotting the LC and SED

In [ ]:
flux_model = open(
    dl3_path / f'{obj_name}_dataset_{datasets[0].name}_to_{datasets[-1].name}_flux_model_dict.dat', 
    'rb'
)
model_dict = pickle.load(flux_model)
flux_model.close()

In [ ]:
flux_points = FluxPoints.read(
    dl3_path / f'{obj_name}_dataset_{datasets[0].name}_to_{datasets[-1].name}_flux_pts.fits',
    hdu="SED",
    format="gadf-sed",
    reference_model=SkyModel.from_dict(model_dict)
)
flux_points_lc = FluxPoints.read(
    dl3_path / f'{obj_name}_dataset_{datasets[0].name}_to_{datasets[-1].name}_flux_pts.fits',
    hdu="LC",
    sed_type="flux", 
    format="lightcurve",
    reference_model=SkyModel.from_dict(model_dict)
)


In [ ]:
plt.figure(figsize=(12,5))

plt.subplot(121)
flux_points_lc.plot(sed_type="flux", **lc_kwargs)
plt.grid(which="both")
plt.title(
    f'LC {flux_points_lc.to_table(sed_type="flux", format="lightcurve")["e_min"].quantity[0][0].to(u.GeV):.2f} < ' +
    f'E < {flux_points_lc.to_table(sed_type="flux", format="lightcurve")["e_max"].quantity[0][0]:.2f}'
)
plt.axhline(
    flux_crab.to_value("cm-2 s-1"), c='red', ls='--', label=ref_label
)
plt.axhspan(
    (flux_crab - flux_crab_error).to_value("cm-2 s-1"), 
    (flux_crab + flux_crab_error).to_value("cm-2 s-1"), 
    alpha=0.2, color='tab:orange'
)


plt.subplot(122)
flux_points.plot(sed_type="e2dnde", **sed_plot_kwargs)
flux_points.reference_model.spectral_model.plot(**sed_kwargs)
flux_points.reference_model.spectral_model.plot_error(**sed_kwargs)
create_crab_spectral_model("magic_lp").plot(
    **sed_magic_ref_kwargs, label=ref_label
)
plt.legend()
plt.grid()
plt.title("SED")